## Masks to Bounding boxes (semantic segmentation to object detection segmentetaion)

In [9]:
# Segmentation classes
list_of_classes = ['Background', 'Water', 'Building_No_Damage', 'Building_Minor_Damage', 
                   'Building_Major_Damage', 'Building_Total_Destruction', 'Vehicle', 'Road-Clear', 
                   'Road-Blocked', 'Tree', 'Pool']

In [1]:
import cv2
import numpy as np
import os
import traceback

In [13]:
def process_image(image_path, image_paths2):
    #print(f"Starting processing {image_path} and {image_paths2}")
    rotated_boxes = []
    try:
        image1 = cv2.imread(image_path)
        image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
        
        class_colors = {
            0: (0, 0, 0), # Background
            1: (255, 0, 0), # Water
            2: (0, 255, 0), # Building_No_Damage
            3: (0, 255, 255), # Building_Minor_Damage
            4: (0, 0, 255), # Building_Major_Damage
            5: (255, 0, 255), # Building_Total_Destruction
            6: (0, 165, 255), # Vehicle
            7: (42, 42, 165), # Road-Clear
            8: (105, 105, 105), # Road-Blocked
            9: (203, 192, 255), # Tree
            10: (255, 255, 0) # Pool
        }
        
        lut = np.zeros((256, 1, 3), dtype=np.uint8)
        for class_id, color in class_colors.items():
            lut[class_id] = color
        colored_image = cv2.LUT(image1, lut)

        image2 = cv2.imread(image_paths2)
        image_width = image2.shape[0]
        image_height = image2.shape[1]
        # Draw bounding boxes
        for class_id, color in class_colors.items():
            if class_id != 0:  
                mask = cv2.inRange(colored_image, np.array(color), np.array(color))
                contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                for contour in contours:
                    rotated_rect = cv2.minAreaRect(contour)
                    rectangle_color = class_colors[class_id]
                    box = cv2.boxPoints(rotated_rect)
                    box = np.int0(box)
                    rotated_boxes.append((class_id, box))
                    for class_id, box in rotated_boxes:
                        rectangle_color = class_colors[class_id]
                        cv2.putText(image2, str(f"class:{class_id}"), (box[0][0], box[0][1]), cv2.FONT_HERSHEY_SIMPLEX, 3, rectangle_color, 3, cv2.LINE_AA)
                    cv2.drawContours(image2, [box], 0, rectangle_color, 12) 

        save_dir = r"M:\SOL\YOLO\Experimantation__Folder\Cnvrtd_train_image" #Save directory
        output_filename = os.path.basename(image_paths2)
        output_path = os.path.join(save_dir, os.path.splitext(output_filename)[0] + '_bbox.jpg')
        cv2.imwrite(output_path, image2)
        # print(f"Finished processing {image_path}")
    except Exception as e:
        print(f"Error processing {image_path} and {image_paths2}: {e}")
        traceback.print_exc()
    box_to_yolo_cords = box_to_yolo_format(rotated_boxes, image_width, image_height)
    return box_to_yolo_cords

def box_to_yolo_format(rotated_boxes, image_width, image_height):
    yolo_coordinates = []
    for r_box in rotated_boxes:
        points = r_box[1]
        points[:, 0] = np.clip(points[:, 0], 0, image_width)
        points[:, 1] = np.clip(points[:, 1], 0, image_height) 
        # Calculate the axis-aligned bounding box
        x_min, y_min = np.min(points, axis=0)
        x_max, y_max = np.max(points, axis=0)

        # Calculate center, width, and height
        x_center = (x_min + x_max) / 2
        y_center = (y_min + y_max) / 2
        width = x_max - x_min
        height = y_max - y_min

        # Normalize the values
        x_center_normalized = x_center / image_width
        y_center_normalized = y_center / image_height
        width_normalized = width / image_width
        height_normalized = height / image_height

        # YOLO format coordinates
        yolo_coordinate = [x_center_normalized, y_center_normalized, width_normalized, height_normalized]
        yolo_coordinates.append(yolo_coordinate)
        return yolo_coordinates

def main():
    image_dir = r"M:\SOL\YOLO\Experimantation__Folder\Experiment_cnvrtng_labeled" #Enter labeled image directory
    image_dir2 = r"M:\SOL\YOLO\Experimantation__Folder\Experiment_cnvrtng_org" #Enter original image directory
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir)]
    image_paths2 = [os.path.join(image_dir2, filename) for filename in os.listdir(image_dir2)]
    combined_paths = list(zip(image_paths, image_paths2))
    print(f"Found {len(combined_paths)} image pairs to process.")
    for image_path, image_paths2 in combined_paths:
        process_image(image_path, image_paths2)
    box_to_cord = process_image(image_path, image_paths2)
    print(box_to_cord[0])
    

if __name__ == '__main__':
    main()

Found 37 image pairs to process.
[0.39116666666666666, 0.261125, 0.463, 0.33625]
